In [ ]:
# ========================================================
# Smoothing + Sharpening Filters Dashboard (Full Assignment Ready)
# ========================================================

# 1. Imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
from IPython.display import display, clear_output
from PIL import Image
from scipy import ndimage as ndi
import datetime

# --------------------------------------------------------
# Utility Functions
# --------------------------------------------------------
def plot_results(original_img, processed_img, processed_title="Processed Image"):
    """Show original and processed images + histograms."""
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    axes[0].imshow(original_img, cmap='gray', vmin=0, vmax=255)
    axes[0].set_title("Original"); axes[0].axis('off')
    axes[1].imshow(processed_img, cmap='gray', vmin=0, vmax=255)
    axes[1].set_title(processed_title); axes[1].axis('off')
    plt.tight_layout(); plt.show()

    fig, axes = plt.subplots(1, 2, figsize=(12, 3))
    axes[0].hist(original_img.ravel(), bins=256, range=(0,255))
    axes[0].set_title("Original Histogram"); axes[0].set_xlim([0,255])
    axes[1].hist(processed_img.ravel(), bins=256, range=(0,255))
    axes[1].set_title("Processed Histogram"); axes[1].set_xlim([0,255])
    plt.tight_layout(); plt.show()

def to_uint8(img):
    img = np.asarray(img)
    if img.dtype == np.uint8:
        return img
    if img.dtype in [np.float32, np.float64]:
        img = np.clip(img, 0, 255)
        return img.astype(np.uint8)
    return np.clip(img, 0, 255).astype(np.uint8)

# --------------------------------------------------------
# Noise Injection
# --------------------------------------------------------
def add_gaussian_noise(img, mean=0, var=0.01):
    sigma = var**0.5
    gauss = np.random.normal(mean, sigma, img.shape)
    noisy = img/255.0 + gauss
    noisy = np.clip(noisy, 0, 1)
    return (noisy*255).astype(np.uint8)

def add_salt_pepper_noise(img, amount=0.02):
    noisy = img.copy()
    num_salt = np.ceil(amount * img.size * 0.5)
    num_pepper = np.ceil(amount * img.size * 0.5)
    coords = [np.random.randint(0, i-1, int(num_salt)) for i in img.shape]
    noisy[tuple(coords)] = 255
    coords = [np.random.randint(0, i-1, int(num_pepper)) for i in img.shape]
    noisy[tuple(coords)] = 0
    return noisy

# --------------------------------------------------------
# Smoothing Filters
# --------------------------------------------------------
def mean_filter(img, ksize=3): return cv2.blur(img, (ksize, ksize))
def median_filter(img, ksize=3):
    if ksize % 2 == 0: ksize += 1
    return cv2.medianBlur(img, ksize)
def _mode_of_window(window):
    vals, counts = np.unique(window, return_counts=True)
    return vals[np.argmax(counts)]
def mode_filter(img, size=3):
    if size % 2 == 0: size += 1
    return ndi.generic_filter(img, function=_mode_of_window, size=(size, size)).astype(np.uint8)

# --------------------------------------------------------
# Sharpening Filters
# --------------------------------------------------------
def sobel_filter(img, ksize=3):
    gx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=ksize)
    gy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=ksize)
    mag = cv2.magnitude(gx, gy)
    return np.clip(mag, 0, 255).astype(np.uint8)

def laplacian_filter(img, ksize=3):
    lap = cv2.Laplacian(img, cv2.CV_64F, ksize=ksize)
    return np.clip(np.absolute(lap), 0, 255).astype(np.uint8)

def sobel_then_laplacian(img, sobel_ksize=3, lap_ksize=3):
    s = sobel_filter(img, ksize=sobel_ksize)
    lap = cv2.Laplacian(s, cv2.CV_64F, ksize=lap_ksize)
    return np.clip(np.absolute(lap), 0, 255).astype(np.uint8)

# --------------------------------------------------------
# Combined
# --------------------------------------------------------
def apply_smoothing_then_sharpening(img, smoothing_name, smoothing_param, sharpening_name, sharpening_param):
    if smoothing_name == "Mean": smooth = mean_filter(img, smoothing_param)
    elif smoothing_name == "Median": smooth = median_filter(img, smoothing_param)
    elif smoothing_name == "Mode": smooth = mode_filter(img, smoothing_param)
    else: smooth = img.copy()

    if sharpening_name == "Sobel": sharp = sobel_filter(smooth, sharpening_param)
    elif sharpening_name == "Laplacian": sharp = laplacian_filter(smooth, sharpening_param)
    elif sharpening_name == "Sobel + Laplacian": sharp = sobel_then_laplacian(smooth, sharpening_param, sharpening_param)
    else: sharp = smooth
    return smooth, sharp

# --------------------------------------------------------
# UI Components
# --------------------------------------------------------
uploader = widgets.FileUpload(accept='image/*', multiple=False, description='Upload Image', layout=Layout(width='100%'))

noise_dropdown = widgets.Dropdown(
    options=["None", "Gaussian", "Salt & Pepper"], value="None",
    description="Noise:", layout=Layout(width='100%')
)

smoothing_dropdown = widgets.Dropdown(
    options=["Mean", "Median", "Mode"], value="Mean",
    description="Smoothing:", layout=Layout(width='100%')
)
smoothing_ksize_slider = widgets.IntSlider(value=3, min=1, max=15, step=2, description="Window:", layout=Layout(width='100%'))
smoothing_apply_button = widgets.Button(description="Apply Smoothing", button_style='info', layout=Layout(width='100%'))

sharpening_dropdown = widgets.Dropdown(
    options=["Sobel", "Laplacian", "Sobel + Laplacian"], value="Sobel",
    description="Sharpening:", layout=Layout(width='100%')
)
sharpening_ksize_slider = widgets.IntSlider(value=3, min=1, max=7, step=2, description="Kernel:", layout=Layout(width='100%'))
sharpening_apply_button = widgets.Button(description="Apply Sharpening", button_style='info', layout=Layout(width='100%'))

combine_checkbox = widgets.Checkbox(value=False, description="Apply smoothing before sharpening", layout=Layout(width='100%'))
combine_apply_button = widgets.Button(description="Apply Combined Pipeline", button_style='success', layout=Layout(width='100%'))

info_area = widgets.Output(layout=Layout(border='solid 1px #cccccc', padding='8px', width='65%'))
save_button = widgets.Button(description="Save Last Result", button_style='warning', layout=Layout(width='100%'))

controls_panel = VBox([
    uploader, noise_dropdown, smoothing_dropdown, smoothing_ksize_slider, smoothing_apply_button,
    sharpening_dropdown, sharpening_ksize_slider, sharpening_apply_button,
    combine_checkbox, combine_apply_button, save_button
], layout=Layout(border='solid 1px #cccccc', padding='10px', width='35%'))

dashboard = HBox([controls_panel, info_area])
display(dashboard)

# --------------------------------------------------------
# State
# --------------------------------------------------------
original_image = None
last_processed_image = None

def load_uploaded_image(uploader_widget):
    if not uploader_widget.value: return None
    uploaded = next(iter(uploader_widget.value.values()))
    arr = np.frombuffer(uploaded['content'], dtype=np.uint8)
    return cv2.imdecode(arr, cv2.IMREAD_GRAYSCALE)

# --------------------------------------------------------
# Event Handlers
# --------------------------------------------------------
def on_smoothing_apply(b):
    global original_image, last_processed_image
    original = load_uploaded_image(uploader)
    if original is None: return
    img = to_uint8(original)
    if noise_dropdown.value == "Gaussian": img = add_gaussian_noise(img)
    elif noise_dropdown.value == "Salt & Pepper": img = add_salt_pepper_noise(img)

    if smoothing_dropdown.value == "Mean": result = mean_filter(img, smoothing_ksize_slider.value)
    elif smoothing_dropdown.value == "Median": result = median_filter(img, smoothing_ksize_slider.value)
    else: result = mode_filter(img, smoothing_ksize_slider.value)

    last_processed_image = result
    with info_area:
        clear_output(wait=True)
        plot_results(img, result, f"{smoothing_dropdown.value} Filter")
        display(widgets.HTML(f"<b>{smoothing_dropdown.value}</b> smoothing applied."))

def on_sharpening_apply(b):
    global original_image, last_processed_image
    original = load_uploaded_image(uploader)
    if original is None: return
    img = to_uint8(original)
    if noise_dropdown.value == "Gaussian": img = add_gaussian_noise(img)
    elif noise_dropdown.value == "Salt & Pepper": img = add_salt_pepper_noise(img)

    if sharpening_dropdown.value == "Sobel": result = sobel_filter(img, sharpening_ksize_slider.value)
    elif sharpening_dropdown.value == "Laplacian": result = laplacian_filter(img, sharpening_ksize_slider.value)
    else: result = sobel_then_laplacian(img, sharpening_ksize_slider.value, sharpening_ksize_slider.value)

    last_processed_image = result
    with info_area:
        clear_output(wait=True)
        plot_results(img, result, f"{sharpening_dropdown.value} Filter")
        display(widgets.HTML(f"<b>{sharpening_dropdown.value}</b> sharpening applied."))

def on_combine_apply(b):
    global last_processed_image
    img = load_uploaded_image(uploader)
    if img is None: return
    img = to_uint8(img)
    if noise_dropdown.value == "Gaussian": img = add_gaussian_noise(img)
    elif noise_dropdown.value == "Salt & Pepper": img = add_salt_pepper_noise(img)

    smooth, sharp = apply_smoothing_then_sharpening(img, smoothing_dropdown.value, smoothing_ksize_slider.value, sharpening_dropdown.value, sharpening_ksize_slider.value)
    last_processed_image = sharp

    with info_area:
        clear_output(wait=True)
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        axes[0].imshow(img, cmap='gray'); axes[0].set_title("Original"); axes[0].axis('off')
        axes[1].imshow(smooth, cmap='gray'); axes[1].set_title("Smoothed"); axes[1].axis('off')
        axes[2].imshow(sharp, cmap='gray'); axes[2].set_title("Sharpened"); axes[2].axis('off')
        plt.show()
        display(widgets.HTML("<b>Combined smoothing + sharpening applied.</b>"))

def on_save(b):
    if last_processed_image is None: return
    filename = f"result_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
    cv2.imwrite(filename, last_processed_image)
    with info_area:
        display(widgets.HTML(f"<p>Saved as {filename}</p>"))

smoothing_apply_button.on_click(on_smoothing_apply)
sharpening_apply_button.on_click(on_sharpening_apply)
combine_apply_button.on_click(on_combine_apply)
save_button.on_click(on_save)

# --------------------------------------------------------
# Notes
# --------------------------------------------------------
display(widgets.HTML("""
<h3>Assignment Notes</h3>
<ul>
<li><b>First-order derivative (Sobel):</b> Highlights edges and gradient direction.</li>
<li><b>Second-order derivative (Laplacian):</b> More sensitive, enhances fine detail, amplifies noise.</li>
<li><b>Combined (Sobel + Laplacian):</b> Produces sharper transitions by combining both.</li>
<li><b>Mean vs Median vs Mode:</b> Median is best for salt-and-pepper, mean for Gaussian, mode preserves dominant background levels.</li>
<li><b>Smoothing before sharpening:</b> Reduces noise amplification, but excessive smoothing weakens edges.</li>
</ul>
"""))
